In [11]:
import gdown
import pandas as pd
import os
import csv
import re

In [12]:
def download_csv():
    sheet_url = "https://docs.google.com/spreadsheets/d/1vYD9nyEoI2CEWu2UY2xpedC_4mX3TUVD-XLCjOLphiA/export?format=csv"

    # Load the CSV into a DataFrame
    df = pd.read_csv(sheet_url)
    df.to_csv("form.csv")
    # print(df)

In [13]:
download_csv()

In [14]:
original_csv = "form.csv"
processed_csv = "processed.csv"
clothes_folder = "clothes_data"
possible_columns = [
    "服裝部位 Clothing Part",
    "上半身 衣服種類 Top Clothing Type",
    "下半身衣服種類 Bottom Clothing Type",
]

In [15]:
df = pd.read_csv(original_csv)
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df2 = df.drop("時間戳記", axis=1)
for col in possible_columns:
    df2 = df2.drop(col, axis=1)

In [ ]:
# get all feature
all_features = []
for index, row in df.iterrows():
    for col in possible_columns:
        str = df.loc[index, col]
        try:
            str = list(str.split(","))
            for s in str:
                s = s.rstrip()
                s = s.lstrip()
                if (
                    s == "針織衣 Sweater"
                    or s == "毛衣 Sweater"
                    or s == "針/織衣 Sweater"
                    or s == "毛衣、針/織衣 Sweater"
                ):
                    s = "毛衣、針織衣 Sweater"
                if s == "西裝褲 dress pants":
                    str.append("褲子 pants")
                    s = "西裝褲、裙 dress pants、skirt"
                all_features.append(s)
        except:
            pass
all_features = sorted(list(set(all_features)))
for f in all_features:
    print(f)
print(len(all_features))

feature_df = pd.DataFrame({"name": all_features})
feature_df.to_csv("feature_list.csv")

Shirt
T-Shirt
上半身 Top
下半身 Bottom
休閒褲/裙 casual pants/skirt
吊帶褲/裙 overalls/jumper skirt
外套 Jacket
工裝褲/裙 cargo pants/skirt
毛衣、針/織衣 Sweater
毛衣、針織衣 Sweater
洋裝 Dress
牛仔材質 Denim
牛仔褲/裙 jeans / demi skirt
百褶裙 pleated skirt
網紗 Mesh
緊身褲 leggings
羽絨衣 Down coat / jacket
背心 Vest
裙子 skirt
褲子 pants
西裝 Suit
西裝褲、裙 dress pants、skirt
連帽衣 Hoodie
運動材質 Sport Tops
運動褲/裙 sweat pants / sports skirt
25


In [27]:
# get add brands
brand_col = "品牌 Brand"
brand_list = sorted(list(set(df[brand_col])))
brand_df = pd.DataFrame({"brand": brand_list})
print(brand_df)
brand_df.to_csv("brand_list.csv")

           brand
0     50 percent
1      AIR SPACE
2         Adidas
3           Caco
4            GAP
5            H&M
6          Lativ
7            Net
8    New Balance
9           Nike
10         PAZZO
11          Puma
12    Queen Shop
13    Timberland
14  Under Armour
15        Uniqlo


In [17]:
for index, row in df.iterrows():
    for fea in all_features:
        has = 0
        for col in possible_columns:
            str = df.loc[index, col]
            try:
                str = list(str.split(","))
                for s in str:
                    s = s.rstrip()
                    s = s.lstrip()
                    if (
                        s == "針織衣 Sweater"
                        or s == "毛衣 Sweater"
                        or s == "針/織衣 Sweater"
                    ):
                        s = "毛衣、針織衣 Sweater"
                    if s == "西裝褲 dress pants":
                        str.append("褲子 pants")
                        s = "西裝褲、裙 dress pants、skirt"
                    if s == fea:
                        has = 1
            except:
                pass
        df2.loc[index, fea] = has


print(df2)

                                             照片 Picture    品牌 Brand  \
0     https://drive.google.com/open?id=1Ki8gALrL6WEy...         H&M   
1     https://drive.google.com/open?id=1rsB7XhHjAww6...         H&M   
2     https://drive.google.com/open?id=1BEUPTgjmZ0uZ...         H&M   
3     https://drive.google.com/open?id=12MW0xvZVVNVo...         H&M   
4     https://drive.google.com/open?id=1rj0PNtlTPp5O...         H&M   
...                                                 ...         ...   
3590  https://drive.google.com/open?id=1X7qZt19nhPIn...  Timberland   
3591  https://drive.google.com/open?id=1LCt2PWB4C4DN...       Lativ   
3592  https://drive.google.com/open?id=1GTS3b5a3obAq...       Lativ   
3593  https://drive.google.com/open?id=1gqY_GU5N9L6A...       Lativ   
3594  https://drive.google.com/open?id=1sMJ2aWpLk0Ve...       Lativ   

     衣服長度 Clothes Length  價格 Price  毛衣、針織衣 Sweater  緊身褲 leggings  \
0                短 short       499             0.0           0.0   
1          

In [18]:
for index, row in df2.iterrows():
    image_url = row["照片 Picture"]  # 從表單中的照片 URL 讀取

    if re.search(r"open\?id=", image_url):
        output = f"image_{index:05}.jpg"
        df2.loc[index, "照片 Picture"] = output
df2.to_csv(processed_csv)

# check image downloaded or not

In [19]:
def download_image(image_url, path):
    file_id = image_url.split("open?id=")[-1]
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    gdown.download(url, path, quiet=False)

In [20]:
for index, row in df2.iterrows():
    output = f"image_{index:05}.jpg"
    path = os.path.join(clothes_folder, output)
    if not os.path.exists(path):
        image_url = df.loc[index, "照片 Picture"]
        print("download ", index, " from ", image_url)
        download_image(image_url, path)